# SQLite Databases
In this notebook we learn basic SQL concepts using a **self-contained SQLite database** built directly
in Python.

We will:

- create a synthetic measurement database  
- practice SELECT, filtering, sorting  
- use aggregation (COUNT, SUM, AVG, …)  
- learn JOINs  
- practice grouping and subqueries  
- explore Common Table Expressions (CTEs)  
- create and modify tables  
- use transactions  

The database contains four tables:

- **Data(X, Y, RunID, ID)** – synthetic measurement points  
- **Runs(RunID, UserID, InstrumentID)** – metadata about measurement runs  
- **Users(UserID, Name)** – synthetic people  
- **Instruments(InsID, Name)** – synthetic instruments  

In [1]:
# Creating the Database
import sqlite3
import numpy as np
import pandas as pd

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Create tables
cur.executescript("""
CREATE TABLE Users (
    UserID INTEGER PRIMARY KEY,
    Name TEXT
);

CREATE TABLE Instruments (
    InsID INTEGER PRIMARY KEY,
    Name TEXT
);

CREATE TABLE Runs (
    RunID INTEGER PRIMARY KEY,
    UserID INTEGER,
    InsID INTEGER
);

CREATE TABLE Data (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    RunID INTEGER,
    X REAL,
    Y REAL
);
""")
conn.commit()

# Insert Users
users = ["Alice", "Bob", "Carol", "Dave"]
cur.executemany("INSERT INTO Users (Name) VALUES (?)", [(u,) for u in users])

# Insert Instruments
instruments = ["Spectro-1", "Spectro-2"]
cur.executemany("INSERT INTO Instruments (Name) VALUES (?)", [(i,) for i in instruments])

# Insert synthetic runs
runs = [
    (1, 1, 1),
    (2, 2, 1),
    (3, 3, 2),
    (4, 4, 2)
]
cur.executemany("INSERT INTO Runs (RunID, UserID, InsID) VALUES (?, ?, ?)", runs)

# Generate synthetic Data table
rng = np.random.default_rng(42)
for run_id in [1, 2, 3, 4]:
    for _ in range(50):
        x = rng.normal(loc=run_id*0.5, scale=1.0)
        y = x + rng.normal(loc=0, scale=0.5)
        cur.execute("INSERT INTO Data (RunID, X, Y) VALUES (?, ?, ?)", (run_id, x, y))

conn.commit()

## First Queries
We start with small, safe queries to peek at the dataset.

In [2]:
# peek at the data
pd.read_sql("SELECT * FROM Data LIMIT 5", conn)

,ID,RunID,X,Y
0,1,1,0.804717,0.284725
1,2,1,1.250451,1.720734
2,3,1,-1.451035,-2.102125
3,4,1,0.627840,0.469719
4,5,1,0.483199,0.056677


In [3]:
# sorting
pd.read_sql("SELECT X, Y FROM Data ORDER BY X LIMIT 5", conn)

,X,Y
0,-1.451035,-2.102125
1,-1.187334,-1.910891
2,-1.182870,-1.350312
3,-1.132046,-0.998191
4,-0.957156,-1.116991


In [4]:
# filtering
pd.read_sql("""
SELECT X, Y
FROM Data
WHERE 2*Y BETWEEN -SIN(X) AND X
ORDER BY RunID DESC, Y DESC
LIMIT 5
""", conn)

,X,Y
0,1.794838,0.793077
1,0.398722,0.001654
2,1.366177,0.678729
3,0.736943,0.180737
4,0.437586,0.172866


## Aggregation
SQL allows summarizing data with aggregates like COUNT, AVG, MAX, and MIN.

In [5]:
# counting rows
pd.read_sql("SELECT COUNT(*) AS N FROM Data", conn)

,N
0,200


In [6]:
# agg and filter
pd.read_sql("""
SELECT COUNT(*) AS N,
       AVG(X) AS MeanX,
       AVG(Y) AS MeanY,
       AVG(Y - X) AS ResidualMean
FROM Data
WHERE Y > 0
""", conn)

,N,MeanX,MeanY,ResidualMean
0,162,1.622032,1.667217,0.045185


In [7]:
# grouping
pd.read_sql("""
SELECT RunID,
       MIN(X) AS MinX,
       MAX(X) AS MaxX,
       MIN(Y) AS MinY,
       MAX(Y) AS MaxY
FROM Data
GROUP BY RunID
ORDER BY MaxX DESC
""", conn)

,RunID,MinX,MaxX,MinY,MaxY
0,3,-0.647289,4.405067,-0.991251,4.955896
1,4,-0.310103,3.820646,-0.282927,4.186034
2,2,-1.132046,3.128470,-0.998191,4.453662
3,1,-1.451035,2.641648,-2.102125,2.438440


## Joins
Joins let us combine information across related tables.

In [8]:
# users with runs
pd.read_sql("""
SELECT Users.Name, Runs.RunID
FROM Runs
JOIN Users ON Users.UserID = Runs.UserID
""", conn)

,Name,RunID
0,Alice,1
1,Bob,2
2,Carol,3
3,Dave,4


In [9]:
# users with no runs
pd.read_sql("""
SELECT Users.Name
FROM Users
LEFT JOIN Runs ON Users.UserID = Runs.UserID
WHERE Runs.RunID IS NULL
""", conn)

,Name


## Subqueries and Nested Queries
Who ran the earliest measurement?

In [10]:
pd.read_sql("""
SELECT Name
FROM Users
WHERE UserID = (
      SELECT UserID
      FROM Runs
      WHERE RunID = (
            SELECT RunID
            FROM Data
            ORDER BY ID
            LIMIT 1
      )
)
""", conn)

,Name
0,Alice


## Common Table Expressions (CTEs)
CTEs allow writing cleaner SQL by naming intermediate results.

In [11]:
# find the two busiest users
pd.read_sql("""
WITH Busy(UserID, N) AS (
    SELECT RunID, COUNT(*) AS N
    FROM Data
    GROUP BY RunID
    ORDER BY N DESC
)
SELECT * FROM Busy LIMIT 2
""", conn)

,UserID,N
0,4,50
1,3,50


In [12]:
# recursive CTE
pd.read_sql("""
WITH RECURSIVE nums(x) AS (
    SELECT 1
    UNION ALL
    SELECT x+1 FROM nums WHERE x < 10
)
SELECT * FROM nums
""", conn)

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


## Data Management

In [13]:
# creating a grades table
cur.execute("""
CREATE TABLE Grades (
    GradeID INTEGER PRIMARY KEY AUTOINCREMENT,
    Grade TEXT UNIQUE,
    ScoreLimit INT
)
""")
conn.commit()

In [14]:
# insert data
grades = [("A+", 90), ("A", 80), ("B", 70), ("C", 60), ("D", 50)]
cur.executemany("INSERT INTO Grades (Grade, ScoreLimit) VALUES (?, ?)", grades)
conn.commit()

## Transactions

In [15]:
try:
    cur.execute("BEGIN TRANSACTION")
    cur.execute("INSERT INTO Instruments (Name) VALUES ('Temp Instrument')")
    raise Exception("Simulated error")
    cur.execute("COMMIT")
except:
    cur.execute("ROLLBACK")

pd.read_sql("SELECT * FROM Instruments", conn)

,InsID,Name
0,1,Spectro-1
1,2,Spectro-2
